In [1]:
# Step 1: Import necessary libraries
import pandas as pd

# Step 2: Load data from CSV files
file_paths = [
    r"C:\Users\USER PC\Downloads\store_sales_1.csv",
    r"C:\Users\USER PC\Downloads\store_sales_2.csv",
    r"C:\Users\USER PC\Downloads\cleaned_store_sales_3.csv"
]


dataframes = [pd.read_csv(path) for path in file_paths]
combined_df = pd.concat(dataframes, ignore_index=True)

# View the first 5 rows
combined_df.head()



,ProductName,Qty,Unit_Price,SaleDate,CurrencyType,CustomerID,StoreID,Total_Price,Total_Price_OMR,Unit_Price_OMR
0,Smith Paper,3.0,10.5,7/13/2024,OMR,9ca482a2-0356-49c1-b5e3-88ae98d1cc2f,Store_A,NaN,NaN,NaN
1,Johnson Screen,NaN,NaN,2/23/2025,Usd,c0b9df4e-8f03-4bf0-a31b-0a7d7c2a8907,Store_A,NaN,NaN,NaN
2,Roberts Ingredient,3.0,30.0,11/13/2024,USD,97dc18e3-2c12-4e26-9863-32514e82e822,Store_A,NaN,NaN,NaN
3,White Monitor,NaN,10.5,4/16/2025,USD,e4d09733-d496-47b3-a4b5-04de84d8fd06,Store_A,NaN,NaN,NaN
4,Rodriguez Keyboard,2.0,20.0,8/3/2024,usd,435ecb46-4545-4af7-b72c-119f64d193a5,Store_A,NaN,NaN,NaN


In [2]:
# Fill missing Qty with 0 
combined_df['Qty'] = combined_df['Qty'].fillna(0).astype(int)

# Fill missing Unit_Price with median price
combined_df['Unit_Price'] = combined_df['Unit_Price'].fillna(combined_df['Unit_Price'].median()).astype(float)

# Fill missing CustomerID with placeholder "Unknown"
combined_df['CustomerID'] = combined_df['CustomerID'].fillna("Unknown")

# Preview the cleaned data
combined_df[['Qty', 'Unit_Price', 'CustomerID']].head()


,Qty,Unit_Price,CustomerID
0,3,10.50,9ca482a2-0356-49c1-b5e3-88ae98d1cc2f
1,0,15.75,c0b9df4e-8f03-4bf0-a31b-0a7d7c2a8907
2,3,30.00,97dc18e3-2c12-4e26-9863-32514e82e822
3,0,10.50,e4d09733-d496-47b3-a4b5-04de84d8fd06
4,2,20.00,435ecb46-4545-4af7-b72c-119f64d193a5


In [12]:
# Step 1: Handle conversion errors
combined_df['Qty'] = pd.to_numeric(combined_df['Qty'], errors='coerce').astype('Int64')
combined_df['Unit_Price'] = pd.to_numeric(combined_df['Unit_Price'], errors='coerce').astype(float)
combined_df['SaleDate'] = pd.to_datetime(combined_df['SaleDate'], errors='coerce')

# Step 2: Check for conversion issues
print("Nulls after conversion:")
print(combined_df[['Qty', 'Unit_Price', 'SaleDate']].isnull().sum())

# Step 3: Drop or handle invalid rows if needed
combined_df = combined_df.dropna(subset=['Qty', 'Unit_Price', 'SaleDate'])

# Step 4: Confirm changes
print("\nFinal Data Types:")
print(combined_df.dtypes)


Nulls after conversion:
Qty            0
Unit_Price     0
SaleDate      58
dtype: int64

Final Data Types:
ProductName                object
Qty                         Int64
Unit_Price                float64
SaleDate           datetime64[ns]
CurrencyType               object
CustomerID                 object
StoreID                    object
Total_Price               float64
Total_Price_OMR           float64
Unit_Price_OMR            float64
dtype: object


In [13]:
# Standardize ProductName and CurrencyType
combined_df['ProductName'] = combined_df['ProductName'].str.strip().str.title()
combined_df['CurrencyType'] = combined_df['CurrencyType'].str.strip().str.upper()

# Check a few values
combined_df[['ProductName', 'CurrencyType']].head()


C:\Users\USER PC\AppData\Local\Temp\ipykernel_32592\3568512217.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df['ProductName'] = combined_df['ProductName'].str.strip().str.title()
C:\Users\USER PC\AppData\Local\Temp\ipykernel_32592\3568512217.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df['CurrencyType'] = combined_df['CurrencyType'].str.strip().str.upper()


,ProductName,CurrencyType
0,Smith Paper,OMR
1,Johnson Screen,USD
2,Roberts Ingredient,USD
3,White Monitor,USD
4,Rodriguez Keyboard,USD


In [5]:
# Currency conversion rates
conversion_rate = {'USD': 0.385, 'OMR': 1.0}

# Apply conversion to Unit_Price
combined_df['Unit_Price_OMR'] = combined_df.apply(
    lambda row: row['Unit_Price'] * conversion_rate.get(row['CurrencyType'], 1.0),
    axis=1
)

# Preview results
combined_df[['Unit_Price', 'CurrencyType', 'Unit_Price_OMR']].head()


,Unit_Price,CurrencyType,Unit_Price_OMR
0,10.50,OMR,10.50000
1,15.75,USD,6.06375
2,30.00,USD,11.55000
3,10.50,USD,4.04250
4,20.00,USD,7.70000


In [6]:
# Calculate total sale price in OMR
combined_df['Total_Price'] = combined_df['Qty'] * combined_df['Unit_Price_OMR']

# Preview final results
combined_df[['Qty', 'Unit_Price_OMR', 'Total_Price']].head()


,Qty,Unit_Price_OMR,Total_Price
0,3,10.50000,31.50
1,0,6.06375,0.00
2,3,11.55000,34.65
3,0,4.04250,0.00
4,2,7.70000,15.40


In [7]:
# Save to CSV file
combined_df.to_csv("transformed_sales_data.csv", index=False)

# Confirmation message
print("Transformed data saved to 'transformed_sales_data.csv'")


Transformed data saved to 'transformed_sales_data.csv'


In [8]:
combined_df

,ProductName,Qty,Unit_Price,SaleDate,CurrencyType,CustomerID,StoreID,Total_Price,Total_Price_OMR,Unit_Price_OMR
0,Smith Paper,3,10.50,7/13/2024,OMR,9ca482a2-0356-49c1-b5e3-88ae98d1cc2f,Store_A,31.5000,NaN,10.50000
1,Johnson Screen,0,15.75,2/23/2025,USD,c0b9df4e-8f03-4bf0-a31b-0a7d7c2a8907,Store_A,0.0000,NaN,6.06375
2,Roberts Ingredient,3,30.00,11/13/2024,USD,97dc18e3-2c12-4e26-9863-32514e82e822,Store_A,34.6500,NaN,11.55000
3,White Monitor,0,10.50,4/16/2025,USD,e4d09733-d496-47b3-a4b5-04de84d8fd06,Store_A,0.0000,NaN,4.04250
4,Rodriguez Keyboard,2,20.00,8/3/2024,USD,435ecb46-4545-4af7-b72c-119f64d193a5,Store_A,15.4000,NaN,7.70000
...,...,...,...,...,...,...,...,...,...,...
253,Line,1,30.00,2024-10-20,NaN,ace52fd8-98ba-42a9-97d3-d659153095a0,STORE_A,30.0000,11.5500,30.00000
254,Actually,3,20.00,2025-02-21,OMR,c0f70ed4-bd8e-4a8f-a2bf-9b134012f5b9,STORE_A,60.0000,23.1000,20.00000
255,Case,1,30.00,2024-08-22,USD,b0552606-1fe5-420f-abf0-a86289c250ba,STORE_A,11.5500,11.5500,11.55000
256,On,1,20.00,2024-09-06,NaN,045081fa-0df3-4904-820b-067b4176fb85,STORE_A,20.0000,7.7000,20.00000


In [ ]:
pip install mysql-connector-python

In [14]:
import mysql.connector

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="root",
    database="DataB"
)
cursor = conn.cursor()


In [15]:
# Products table
cursor.execute("""
CREATE TABLE IF NOT EXISTS Products (
    ProductID INT AUTO_INCREMENT PRIMARY KEY,
    ProductName VARCHAR(255) UNIQUE
)
""")

# Customers table
cursor.execute("""
CREATE TABLE IF NOT EXISTS Customers (
    CustomerID VARCHAR(50) PRIMARY KEY
)
""")

# Stores table
cursor.execute("""
CREATE TABLE IF NOT EXISTS Stores (
    StoreID VARCHAR(50) PRIMARY KEY
)
""")

# Sales table
cursor.execute("""
CREATE TABLE IF NOT EXISTS Sales (
    SaleID INT AUTO_INCREMENT PRIMARY KEY,
    SaleDate DATE,
    ProductID INT,
    CustomerID VARCHAR(50),
    StoreID VARCHAR(50),
    Qty INT,
    Unit_Price_OMR FLOAT,
    Total_Price FLOAT,
    FOREIGN KEY (ProductID) REFERENCES Products(ProductID),
    FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID),
    FOREIGN KEY (StoreID) REFERENCES Stores(StoreID)
)
""")

conn.commit()


In [16]:
# Insert unique Products
for product in combined_df['ProductName'].drop_duplicates():
    cursor.execute("INSERT IGNORE INTO Products (ProductName) VALUES (%s)", (product,))

# Insert unique Customers
for customer in combined_df['CustomerID'].drop_duplicates():
    cursor.execute("INSERT IGNORE INTO Customers (CustomerID) VALUES (%s)", (customer,))

# Insert unique Stores
for store in combined_df['StoreID'].drop_duplicates():
    cursor.execute("INSERT IGNORE INTO Stores (StoreID) VALUES (%s)", (store,))

conn.commit()


In [17]:
# Build mapping from ProductName to ProductID
cursor.execute("SELECT ProductID, ProductName FROM Products")
product_map = {name: pid for pid, name in cursor.fetchall()}

# Insert each row into Sales
for _, row in combined_df.iterrows():
    product_id = product_map.get(row['ProductName'])

    cursor.execute("""
        INSERT INTO Sales (
            SaleDate, ProductID, CustomerID, StoreID,
            Qty, Unit_Price_OMR, Total_Price
        ) VALUES (%s, %s, %s, %s, %s, %s, %s)
    """, (
        row['SaleDate'], product_id, row['CustomerID'], row['StoreID'],
        row['Qty'], row['Unit_Price_OMR'], row['Total_Price']
    ))

conn.commit()
print("Cleaned data successfully inserted into all tables.")


Cleaned data successfully inserted into all tables.


In [18]:
import pandas as pd

# Display Products
cursor.execute("SELECT * FROM Products")
products_df = pd.DataFrame(cursor.fetchall(), columns=['ProductID', 'ProductName'])
print("Products Table:")
display(products_df)

# Display Customers
cursor.execute("SELECT * FROM Customers")
customers_df = pd.DataFrame(cursor.fetchall(), columns=['CustomerID'])
print("Customers Table:")
display(customers_df)

# Display Stores
cursor.execute("SELECT * FROM Stores")
stores_df = pd.DataFrame(cursor.fetchall(), columns=['StoreID'])
print("Stores Table:")
display(stores_df)

# Display Sales
cursor.execute("""
SELECT SaleID, SaleDate, ProductID, CustomerID, StoreID, Qty, Unit_Price_OMR, Total_Price
FROM Sales
""")
sales_df = pd.DataFrame(cursor.fetchall(), columns=[
    'SaleID', 'SaleDate', 'ProductID', 'CustomerID',
    'StoreID', 'Qty', 'Unit_Price_OMR', 'Total_Price'
])
print("Sales Table:")
display(sales_df)


Products Table:


,ProductID,ProductName
0,24,Adams Brush
1,85,Alexander Spray
2,17,Allen Notebook
3,9,Anderson Lamp
4,139,Arnold Fork
...,...,...
136,4,White Monitor
137,11,Williams Bottle
138,66,Wood Tire
139,96,Woods Detergent


Customers Table:


,CustomerID
0,0104ec48-84a4-4638-9ed4-dad335d44c43
1,0437bab5-0dae-422e-9d2d-719091ea2dd9
2,05d39b68-833c-45d0-9af3-c0b3bad43a5c
3,05d75424-135a-40ea-9f43-7297d527e0e0
4,07991be5-0500-4bdc-95bc-a3ee50f4d268
...,...
174,fe6e6f78-bdc3-4deb-a6a2-5f9cd36d2484
175,feaeae49-e208-4b0b-a4ee-88ab5eb6a7c5
176,ff1f1a3e-4889-46d4-824e-d92e4c51cf45
177,ff8159fe-b68a-4bd9-848d-64b836a96eae


Stores Table:


,StoreID
0,Store_A
1,store-A


Sales Table:


,SaleID,SaleDate,ProductID,CustomerID,StoreID,Qty,Unit_Price_OMR,Total_Price
0,1,2024-07-13,1,9ca482a2-0356-49c1-b5e3-88ae98d1cc2f,Store_A,3,10.50000,31.5000
1,2,2025-02-23,2,c0b9df4e-8f03-4bf0-a31b-0a7d7c2a8907,Store_A,0,6.06375,0.0000
2,3,2024-11-13,3,97dc18e3-2c12-4e26-9863-32514e82e822,Store_A,3,11.55000,34.6500
3,4,2025-04-16,4,e4d09733-d496-47b3-a4b5-04de84d8fd06,Store_A,0,4.04250,0.0000
4,5,2024-08-03,5,435ecb46-4545-4af7-b72c-119f64d193a5,Store_A,2,7.70000,15.4000
...,...,...,...,...,...,...,...,...
195,196,2024-07-27,137,72a815bf-d3e6-444a-8467-61f4157b6bc2,store-A,1,4.04250,4.0425
196,197,2025-05-23,138,9d384907-7e3e-46cc-b11b-40a52a0b6c85,store-A,0,7.70000,0.0000
197,198,2024-09-26,139,78c4328d-f82b-4402-b617-aa92f2cca99d,store-A,2,15.75000,31.5000
198,199,2024-11-07,140,c7a97c12-67e2-408b-bebe-686e875e2e7b,store-A,0,20.00000,0.0000


In [19]:
#Question 1: Top 5 Products by Total Revenue

query1 = """
SELECT 
    P.ProductName, 
    SUM(S.Total_Price) AS TotalRevenue
FROM Sales S
JOIN Products P ON S.ProductID = P.ProductID
GROUP BY P.ProductName
ORDER BY TotalRevenue DESC
LIMIT 5;
"""

top_products_df = pd.read_sql(query1, conn)
print("Top 5 Products by Revenue:")
display(top_products_df)


Top 5 Products by Revenue:


C:\Users\USER PC\AppData\Local\Temp\ipykernel_32592\1138219285.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  top_products_df = pd.read_sql(query1, conn)


,ProductName,TotalRevenue
0,Griffin Cream,108.19125
1,Moore Chair,108.19125
2,Gray Tool,96.06375
3,Ball Notebook,90.00000
4,Anderson Lamp,90.00000


In [20]:
#Question 2: Top 5 Customers by Spending

query2 = """
SELECT 
    C.CustomerID,
    SUM(S.Total_Price) AS TotalSpending
FROM Sales S
JOIN Customers C ON S.CustomerID = C.CustomerID
GROUP BY C.CustomerID
ORDER BY TotalSpending DESC
LIMIT 5;
"""

top_customers_df = pd.read_sql(query2, conn)
print("Top 5 Customers by Spending:")
display(top_customers_df)


Top 5 Customers by Spending:


C:\Users\USER PC\AppData\Local\Temp\ipykernel_32592\943619098.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  top_customers_df = pd.read_sql(query2, conn)


,CustomerID,TotalSpending
0,Unknown,387.406251
1,d9966648-3b63-487c-9011-d52513c114d7,90.000000
2,ca1fa71a-cb7a-49d0-8784-70a46c5a917d,90.000000
3,0e57e6c8-d5c5-43fa-919e-3640f6ebba01,90.000000
4,07991be5-0500-4bdc-95bc-a3ee50f4d268,90.000000


In [21]:
#Question 3: Top 5 Stores by Sales Revenue

query3 = """
SELECT 
    St.StoreID,
    SUM(S.Total_Price) AS StoreRevenue
FROM Sales S
JOIN Stores St ON S.StoreID = St.StoreID
GROUP BY St.StoreID
ORDER BY StoreRevenue DESC
LIMIT 5;
"""

top_stores_df = pd.read_sql(query3, conn)
print("Top 5 Stores by Revenue:")
display(top_stores_df)


Top 5 Stores by Revenue:


C:\Users\USER PC\AppData\Local\Temp\ipykernel_32592\2051731027.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  top_stores_df = pd.read_sql(query3, conn)


,StoreID,StoreRevenue
0,Store_A,1889.973753
1,store-A,1501.528751


In [22]:
!pip install schedule

In [ ]:
import pandas as pd
import mysql.connector
import schedule
import time

# Step 1: Define the OLAP job
def weekly_job():
    print("Running weekly OLAP summary...")

    # Connect to MySQL
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="root",
        database="DataB"
    )

    # OLAP Query: Weekly sales summary
    query = """
    SELECT 
        StoreID,
        YEAR(SaleDate) AS Year,
        WEEK(SaleDate) AS Week,
        SUM(Total_Price) AS TotalSales,
        COUNT(*) AS Transactions
    FROM Sales
    GROUP BY StoreID, YEAR(SaleDate), WEEK(SaleDate)
    ORDER BY Year, Week;
    """

    # Load summary
    weekly_summary = pd.read_sql(query, conn)

    # Export to CSV 
    weekly_summary.to_csv("weekly_sales_olap.csv", index=False)
    print("OLAP summary exported to weekly_sales_olap.csv")

# Step 2: Schedule the job every Tuesday at 2:00 PM
schedule.every().tuesday.at("14:00").do(weekly_job)

# Step 3: Start the scheduler
print("Scheduler is running.")
while True:
    schedule.run_pending()
    time.sleep(60)


In [24]:
import schedule
import time

def say_hello():
    print("Hello! It's Itaa")

schedule.every(10).seconds.do(say_hello)

while True:
    schedule.run_pending()
    time.sleep(1)


Hello! It's Itaa
Hello! It's Itaa
Hello! It's Itaa
Hello! It's Itaa
Hello! It's Itaa
Hello! It's Itaa
Hello! It's Itaa
Hello! It's Itaa
Hello! It's Itaa


KeyboardInterrupt: 

In [25]:
import schedule
import time

def say_hello():
    print("Itaa Is The Best :) ")

# Schedule the job every 10 seconds
schedule.every(10).seconds.do(say_hello)

print("Scheduler started. Press Ctrl+C to stop.")
while True:
    schedule.run_pending()
    time.sleep(1)


Scheduler started. Press Ctrl+C to stop.
Hello! It's Itaa
Hello! It's Itaa
Itaa Is The Best :) 
Hello! It's Itaa
Itaa Is The Best :) 
Hello! It's Itaa
Itaa Is The Best :) 
Hello! It's Itaa
Itaa Is The Best :) 
Hello! It's Itaa
Itaa Is The Best :) 
Hello! It's Itaa
Itaa Is The Best :) 
Hello! It's Itaa
Itaa Is The Best :) 


KeyboardInterrupt: 